In [1]:
import pandas as pd
import numpy as np 
import plotly.express as px

In [2]:
# Read data 

df = pd.read_csv("soccer_results_02_21.csv")
print(df.head())

     Round        Date   Time         Home_Team  Home_Score  Away_Score  \
0  ROUND 1  31/08/2002  22:30  RACING SANTANDER         0.0         1.0   
1  ROUND 1  01/09/2002  21:00    RAYO VALLECANO         2.0         2.0   
2  ROUND 1  01/09/2002  21:00     REAL SOCIEDAD         4.0         2.0   
3  ROUND 1  01/09/2002  22:00          MALLORCA         0.0         2.0   
4  ROUND 1  01/09/2002  22:30        VILLARREAL         2.0         2.0   

         Away_Team  Home_Score_AET  Away_Score_AET  Home_Penalties  \
0       VALLADOLID             NaN             NaN             NaN   
1           ALAVES             NaN             NaN             NaN   
2  ATHLETIC BILBAO             NaN             NaN             NaN   
3         VALENCIA             NaN             NaN             NaN   
4          OSASUNA             NaN             NaN             NaN   

   Away_Penalties  Home_Points  Away_Points  season Country       Competition  
0             NaN          0.0          3.0    2

In [3]:
# Check the size of the dataframe

print("Total results: ", df.shape[0]) # rows
print("Total categories: ", df.shape[1]) # columns (categories)

Total results:  51402
Total categories:  16


In [4]:
# Check the categories and their data types

print(df.dtypes)

Round              object
Date               object
Time               object
Home_Team          object
Home_Score        float64
Away_Score        float64
Away_Team          object
Home_Score_AET    float64
Away_Score_AET    float64
Home_Penalties    float64
Away_Penalties    float64
Home_Points       float64
Away_Points       float64
season              int64
Country            object
Competition        object
dtype: object


In [5]:
# Drop AET columns (I only want to have league games)

df = df.drop(['Home_Score_AET', 'Away_Score_AET'], axis=1)

In [6]:
# Check if Competition include only league games 

print(df['Competition'].unique())

['primera-division' 'premier-league' 'bundesliga' 'ligue-1' 'serie-a'
 'uefa-champions-league' 'uefa-europa-league' 'fa-cup']


In [7]:
# Drop competitions that are not national leagues

df = df[df.Competition != 'fa-cup']
df = df[df.Competition != 'uefa-europa-league']
df = df[df.Competition != 'uefa-champions-league']

# Alternative ways

# df = df[~df['Competition'].isin(['fa-cup', 'uefa-europa-league', 'uefa-champions-league'])]
# df = df.query("Competition not in ['fa-cup', 'uefa-europa-league', 'uefa-champions-league']")

In [8]:
# Check if it includes season 2022 

print(df['season'].unique())

[2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015
 2016 2017 2018 2019 2020 2021 2022]


In [9]:
# Drop season 2022 because it's incomplete

df = df[df.season != 2022]

In [10]:
# Rearange columns 

df = df[['season', 'Country', 'Competition', 'Round', 'Date', 'Time', 'Home_Team', 'Home_Score', 
'Away_Score','Away_Team', 'Home_Penalties', 'Away_Penalties', 'Home_Points', 'Away_Points']]

print(df.columns)

Index(['season', 'Country', 'Competition', 'Round', 'Date', 'Time',
       'Home_Team', 'Home_Score', 'Away_Score', 'Away_Team', 'Home_Penalties',
       'Away_Penalties', 'Home_Points', 'Away_Points'],
      dtype='object')


In [11]:
# Fill Nan's and convert points, scores, and penalties to integer

columns_to_fill = ['Home_Points', 'Away_Points', 'Home_Score', 'Away_Score', 
                   'Home_Penalties', 'Away_Penalties']

for col in columns_to_fill:
    df[col] = df[col].fillna(0).astype(int)

print(df.dtypes)

season             int64
Country           object
Competition       object
Round             object
Date              object
Time              object
Home_Team         object
Home_Score         int64
Away_Score         int64
Away_Team         object
Home_Penalties     int64
Away_Penalties     int64
Home_Points        int64
Away_Points        int64
dtype: object


In [12]:
# Check missing rows in columns

c=df.isnull().sum()
print(c)

season            0
Country           0
Competition       0
Round             0
Date              0
Time              0
Home_Team         0
Home_Score        0
Away_Score        0
Away_Team         0
Home_Penalties    0
Away_Penalties    0
Home_Points       0
Away_Points       0
dtype: int64


In [13]:
# Create new columns - Home_Win, Draw, and Away_Win and set the values

df['Home_Win'] = np.where(df['Home_Points'] == 3, True, False)
df['Draw'] = np.where(df['Home_Points'] == 1, True, False)
df['Away_Win'] = np.where(df['Away_Points'] == 3, True, False)

# Check if it's correct

print(df.head())

   season Country       Competition    Round        Date   Time  \
0    2002   spain  primera-division  ROUND 1  31/08/2002  22:30   
1    2002   spain  primera-division  ROUND 1  01/09/2002  21:00   
2    2002   spain  primera-division  ROUND 1  01/09/2002  21:00   
3    2002   spain  primera-division  ROUND 1  01/09/2002  22:00   
4    2002   spain  primera-division  ROUND 1  01/09/2002  22:30   

          Home_Team  Home_Score  Away_Score        Away_Team  Home_Penalties  \
0  RACING SANTANDER           0           1       VALLADOLID               0   
1    RAYO VALLECANO           2           2           ALAVES               0   
2     REAL SOCIEDAD           4           2  ATHLETIC BILBAO               0   
3          MALLORCA           0           2         VALENCIA               0   
4        VILLARREAL           2           2          OSASUNA               0   

   Away_Penalties  Home_Points  Away_Points  Home_Win   Draw  Away_Win  
0               0            0            3

In [15]:
# Find the the top 10 teams with the most home wins

grouped_h = df.groupby('Home_Team')
home_wins = grouped_h['Home_Win'].sum()
home_wins.sort_values(ascending=False, inplace=True)

print(home_wins.head(10))


Home_Team
BARCELONA              293.0
REAL MADRID            289.0
BAYERN MUNICH          265.0
MANCHESTER UTD         263.0
JUVENTUS               261.0
ARSENAL                254.0
CHELSEA                250.0
MANCHESTER CITY        248.0
PARIS SAINT GERMAIN    244.0
LIVERPOOL              242.0
Name: Home_Win, dtype: float64


In [16]:
# Find the top 10 teams with the most away wins

grouped_a = df.groupby('Away_Team')
away_wins = grouped_a['Away_Win'].sum()
away_wins.sort_values(ascending=False, inplace=True)

print(away_wins.head(10))

Away_Team
REAL MADRID       225.0
BARCELONA         220.0
CHELSEA           208.0
BAYERN MUNICH     207.0
JUVENTUS          200.0
MANCHESTER UTD    196.0
INTER             181.0
AC MILAN          179.0
LIVERPOOL         178.0
LYON              174.0
Name: Away_Win, dtype: float64


In [17]:
# Find the top 10 teams with the most wins in total

total_wins = home_wins.add(away_wins, fill_value=0)
total_wins.sort_values(ascending=False, inplace=True)

print(total_wins.head(10))

REAL MADRID        514.0
BARCELONA          513.0
BAYERN MUNICH      472.0
JUVENTUS           461.0
MANCHESTER UTD     459.0
CHELSEA            458.0
ARSENAL            424.0
INTER              422.0
LIVERPOOL          420.0
MANCHESTER CITY    420.0
dtype: float64


In [18]:
# Find the top 10 teams that scored most goals

home_score = grouped_h['Home_Score'].sum()
away_score = grouped_a['Away_Score'].sum()

total_score = home_score.add(away_score, fill_value=0)
total_score.sort_values(ascending=False, inplace=True)

print(total_score.head(10))

Home_Team
BARCELONA              1826
REAL MADRID            1772
BAYERN MUNICH          1623
MANCHESTER CITY        1437
ARSENAL                1419
CHELSEA                1404
MANCHESTER UTD         1395
LIVERPOOL              1378
PARIS SAINT GERMAIN    1375
INTER                  1352
dtype: int64


In [19]:
# Create function to count the percentage of home wins, draws and away wins 

def calculate_percentage(group):
    total_games = len(group)
    home_wins = group['Home_Win'].eq(True).sum()
    draws = group['Draw'].eq(True).sum()
    away_wins = group['Away_Win'].eq(True).sum()
    
    home_wins_percentage = (home_wins / total_games) * 100
    draws_percentage = (draws / total_games) * 100
    away_wins_percentage = (away_wins / total_games) * 100
    
    return pd.Series({'Home Wins': home_wins_percentage,
                      'Draws': draws_percentage,
                      'Away Wins': away_wins_percentage})

In [20]:
# Grup by season and apply percentage function

per_season = df.groupby('season')
per_season = per_season.apply(calculate_percentage)
per_season.reset_index(level=['season'], inplace=True)

print(per_season)

    season  Home Wins      Draws  Away Wins
0     2002  47.716895  26.826484  25.456621
1     2003  46.575342  26.484018  26.940639
2     2004  46.987952  29.079956  23.932092
3     2005  45.509310  27.601314  26.889376
4     2006  46.111720  27.710843  26.177437
5     2007  46.166484  26.998905  26.834611
6     2008  47.152245  25.246440  27.601314
7     2009  48.028478  26.067908  25.903614
8     2010  46.714129  26.286966  26.998905
9     2011  46.549836  26.560789  26.889376
10    2012  45.564074  25.958379  28.477547
11    2013  46.604600  23.384447  30.010953
12    2014  44.961665  25.958379  29.079956
13    2015  44.304491  25.903614  29.791895
14    2016  48.630887  23.110624  28.258488
15    2017  45.345016  24.479737  30.175246
16    2018  44.742607  25.848850  29.408543
17    2019  41.675794  23.001095  29.791895
18    2020  39.923330  25.410734  34.665936
19    2021  42.747674  25.889436  31.362890


In [21]:
# Plot bar chart to compare seasons

fig = px.bar(per_season, x='season', y=["Home Wins", "Draws", "Away Wins"], 
            title="Home Wins, Draws, and Away Wins from 2002 - 2021 by Season")
fig.show()

In [22]:
# Plot line graph to see trends

fig = px.line(per_season, x="season", y=["Home Wins", "Draws", "Away Wins"],
              title="Home Wins, Draws, and Away Wins from 2002 - 2021 by Season")
fig.show()

In [23]:
# Group by competition and apply percentages

per_comp = df.groupby(['Competition']) 
per_comp = per_comp.apply(calculate_percentage) 
per_comp.reset_index(level=['Competition'], inplace=True)

print(per_comp)

        Competition  Home Wins      Draws  Away Wins
0        bundesliga  45.588235  24.771242  29.640523
1           ligue-1  44.533614  27.851598  26.286015
2    premier-league  45.894737  24.763158  29.342105
3  primera-division  46.907895  25.052632  28.039474
4           serie-a  45.034890  26.798175  28.166935


In [24]:
# Plot bar to compare national leagues

fig = px.bar(per_comp, x='Competition', y=["Home Wins", "Draws", "Away Wins"], 
            title="Home Wins, Draws, and Away Wins from 2002 - 2021 by Competition")
fig.show()